In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import tensorflow as tf
import timeit
import os
from six.moves import cPickle
from text_utils import TextLoader
from tensorflow.contrib import rnn
from char_rnn_model import Model
import numpy as np

# Define directories, hyperparameter

In [3]:
# data locations, etc
data_dir = '.'

# training
num_epochs = 5
batch_size = 512

# network architecture
num_neurons = 256
seq_length = 50
vocab_size = 67
num_layers = 2 
dropout = False
learning_rate = 0.001


# Load data using TextLoader object

In [4]:
loader = TextLoader('.', 20, 50)
#loader.preprocess_data()

# Create your model object

In [10]:
char_model = Model(num_neurons, seq_length, vocab_size, num_layers=3, lr=0.001, use_dropout=True, use_gpu=True)

## Save

In [11]:
saver = tf.train.Saver()

# Training
## Training is done via slurm job on Hyak P100
## Results can be examined in:
## slurm-869469.out

In [ ]:
init = tf.global_variables_initializer()
i = 0
with tf.Session() as sess:
    sess.run(init)
    for epochs in range(num_epochs):
        print("Training for Epoch Number: \t" + str(epochs))
        for num in range(5):
            print("Loading Data...")
            x_train = loader.load_train_data(num)
            print("Batching Data on:\t " + str(num))
            batcher = loader.training_batch(batch_size)
            print("Beginning Training:")
            for data, labels in batcher:
                print(data.shape)
                i+=1
                f_dict = {char_model.X:data[:-1], char_model.Y:data[1:]}
                sess.run(char_model.trainer, feed_dict=f_dict)
                #test = sess.run(char_model.outputs, feed_dict=f_dict)
                #print(test.shape)
                if i % 1 ==0:
                    c_loss = sess.run(char_model.loss, feed_dict=f_dict)
                    print(c_loss)
        save_path = saver.save(sess, "./tmp/model" + str(epochs) + str(num+15) + ".ckpt")
        print("Model saved in path: %s" % save_path)


Training for Epoch Number: 	0
Loading Data...
Batching Data on:	 0
Beginning Training:
(512, 50, 67)
4.199327
(512, 50, 67)
4.1913157
(512, 50, 67)
4.1800666
(512, 50, 67)
4.155279
(512, 50, 67)
4.1134667
(512, 50, 67)
4.0036426
(512, 50, 67)
3.7244954
(512, 50, 67)
3.6742742
(512, 50, 67)
3.5836692
(512, 50, 67)
3.4404502
(512, 50, 67)
3.5961833
(512, 50, 67)
3.383543
(512, 50, 67)
3.5070994
(512, 50, 67)
3.4058828
(512, 50, 67)
3.4388597
(512, 50, 67)
3.3758035
(512, 50, 67)
3.389394
(512, 50, 67)
3.3973327
(512, 50, 67)
3.3220975
(512, 50, 67)
3.5589917
(512, 50, 67)
3.4527612
(512, 50, 67)
3.4907775
(512, 50, 67)
3.641305
(512, 50, 67)
3.3651736
(512, 50, 67)
3.3839407
(512, 50, 67)
3.2883797
(512, 50, 67)
3.3301847
(512, 50, 67)
3.3564317
(512, 50, 67)
3.3342998
(512, 50, 67)
3.1910374
(512, 50, 67)
3.3063033
(512, 50, 67)
3.398341
(512, 50, 67)
3.5531979
(512, 50, 67)
3.4118197
(512, 50, 67)
3.4141133
(512, 50, 67)
3.4988472
(512, 50, 67)
3.4527202
(512, 50, 67)
3.3094177
(512, 5

## Restoring and Text Generation

In [17]:
# load in some val data to test restore
x_val = np.load('s_val.npy')
# load in the character mapper dictionary
vocab = loader.load_vocab()

In [88]:
# helper functions - could have been defined in class
def one_hot_2_string(input_onehot, vocab_dict):
    """
    quick function to process output of rnn and map to characters
    
    """
    s = ""
    for row in np.reshape(input_onehot, (50, 67)):
        for key, val in vocab_dict.items():
            if np.argmax(row) == val:
                s += key
    return s

def string_2_one_hot(input_string, vocab_dict):
    """
    quick function to change string to one-hot encode via vocab dictionary
    """
    encode = []
    for char in input_string:
        encode.append(np.eye(len(vocab))[vocab_dict[char]])
    encode_np = np.array(encode)
    x,y = encode_np.shape
    return np.reshape(encode_np, (1, x, y))

In [96]:
# here we restore the model, and generate text

# get some text to feed in
starter = "First Citizen:  Before we proceed any further, hea"

# map to one-hot
encoded_starter = string_2_one_hot(starter, vocab)
num_char = 500 # modify to generate more text

init = tf.global_variables_initializer()
with tf.Session() as sess:
    # restore the session
    saver = tf.train.import_meta_graph('./tmp/model_0244.ckpt.meta')
    saver.restore(sess, './tmp/model_0244.ckpt')
    for num_char in range(num_char):
        # feed in the last 50 chars
        outputs = sess.run(char_model.logits, feed_dict={char_model.X: string_2_one_hot(starter[-50:], vocab)})
        output_encode = one_hot_2_string(outputs, vocab)
        # append next prediction to output
        starter += output_encode[-1] # last character of predicted output
    print('New Generated Shakespeare Text!!! \n')
    print(starter)

INFO:tensorflow:Restoring parameters from ./tmp/model_0244.ckpt
New Generated Shakespeare Text!!! 

First Citizen:  Before we proceed any further, hear me,
And therefore they are all the state of the state,
And shall be so far as I am so south and search to thee and stand
The state of the sea and the start of the sea,
And then the strong and sound of the seas and straight,
And therefore shall I see the country to the state.

CADE:
Ay, and the commons shall be so many
the star and the stars of the state of the soldiers,
And that the state of the shall be a soldier,
And then the seal of the state of the story
I have a son and the streets of the


# Generating Text ^^ see above!